In [460]:
import pandas as pd

In [461]:
df = pd.read_json('data.json')

In [462]:
df.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [463]:
df.loc[df['highway_cost'] > 0]

,order_id,warehouse_name,highway_cost,products


In [464]:
df['highway_cost'] = abs(df['highway_cost'])

In [465]:
df.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


## Тарифы складов

In [466]:
def get_quantity(products: list[dict]) -> int:
    return sum([i['quantity'] for i in products])

In [467]:
unique_warehouse = df.drop_duplicates(subset='warehouse_name')

In [468]:
unique_warehouse

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,50,"[{'product': 'зеленая пластинка', 'price': 10,..."
5,87044,остров невезения,15,"[{'product': 'плюмбус', 'price': 250, 'quantit..."
6,58598,гиперборея,160,"[{'product': 'плюмбус', 'price': 250, 'quantit..."


In [469]:
tarifs = unique_warehouse['highway_cost'] / unique_warehouse['products'].apply(get_quantity)

In [470]:
d = zip(unique_warehouse['warehouse_name'], tarifs)

In [471]:
answer1_dict = dict(d)

In [472]:
answer1_dict

{'Мордор': 10.0,
 'хутор близ Диканьки': 15.0,
 'отель Лето': 25.0,
 'остров невезения': 5.0,
 'гиперборея': 20.0}

In [473]:
answer1 = pd.DataFrame({
    'warehouse_name': unique_warehouse['warehouse_name'],
    'tarif_cost': tarifs
}
)

### Ответ

In [474]:
answer1

,warehouse_name,tarif_cost
0,Мордор,10.0
1,хутор близ Диканьки,15.0
2,отель Лето,25.0
5,остров невезения,5.0
6,гиперборея,20.0


## Суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара

доходом с товара является цена продажи * количество товара

расходом является тариф для данного склада * количество товара

прибылью является доход - расход

In [475]:
df.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [476]:
answer1_dict[df.iloc[0]['warehouse_name']]

10.0

In [477]:
hash_quantity = {}
hash_income = {}
hash_expenses = {}
hash_profit = {}

In [478]:
for i, products in enumerate(df['products']):
    for product in products:
        name, price, quantity = product['product'], product['price'], product['quantity']

        if name not in hash_quantity:
            hash_quantity[name] = quantity
        else:
            hash_quantity[name] += quantity
            
        if name not in hash_income:
            hash_income[name] = quantity * price
        else:
            hash_income[name] += quantity * price

In [479]:
for i, products in enumerate(df['products']):
    for product in products:
        name, price, quantity = product['product'], product['price'], product['quantity']
        
        if name not in hash_expenses:
            hash_expenses[name] = answer1_dict[df.iloc[i]['warehouse_name']] * hash_quantity[name]
        else:
            hash_expenses[name] += answer1_dict[df.iloc[i]['warehouse_name']] * hash_quantity[name]
    

In [480]:
hash_quantity

{'ломтик июльского неба': 42,
 'билет в Израиль': 58,
 'статуэтка Ленина': 68,
 'зеленая пластинка': 61,
 'автограф Стаса Барецкого': 48,
 'плюмбус': 65,
 'подписка на suppi-блог': 33}

In [481]:
hash_income

{'ломтик июльского неба': 18900,
 'билет в Израиль': 58000,
 'статуэтка Ленина': 13600,
 'зеленая пластинка': 610,
 'автограф Стаса Барецкого': 28800,
 'плюмбус': 16250,
 'подписка на suppi-блог': 4950}

In [482]:
hash_expenses

{'ломтик июльского неба': 14910.0,
 'билет в Израиль': 29000.0,
 'статуэтка Ленина': 35360.0,
 'зеленая пластинка': 29280.0,
 'автограф Стаса Барецкого': 21840.0,
 'плюмбус': 31200.0,
 'подписка на suppi-блог': 10560.0}

In [483]:
unique_products = hash_quantity.keys()

In [484]:
unique_products

dict_keys(['ломтик июльского неба', 'билет в Израиль', 'статуэтка Ленина', 'зеленая пластинка', 'автограф Стаса Барецкого', 'плюмбус', 'подписка на suppi-блог'])

In [485]:
for i in unique_products:
    hash_profit[i] = hash_income[i] - hash_expenses[i]
    

In [486]:
hash_profit

{'ломтик июльского неба': 3990.0,
 'билет в Израиль': 29000.0,
 'статуэтка Ленина': -21760.0,
 'зеленая пластинка': -28670.0,
 'автограф Стаса Барецкого': 6960.0,
 'плюмбус': -14950.0,
 'подписка на suppi-блог': -5610.0}

In [487]:
answer2_dict = {
    'product': unique_products,
    'quantity': hash_quantity.values(),
    'income': hash_income.values(),
    'expenses': hash_expenses.values(),
    'profit': hash_profit.values()
}

In [488]:
answer_2 = pd.DataFrame(answer2_dict)

### Ответ

In [489]:
answer_2

,product,quantity,income,expenses,profit
0,ломтик июльского неба,42,18900,14910.0,3990.0
1,билет в Израиль,58,58000,29000.0,29000.0
2,статуэтка Ленина,68,13600,35360.0,-21760.0
3,зеленая пластинка,61,610,29280.0,-28670.0
4,автограф Стаса Барецкого,48,28800,21840.0,6960.0
5,плюмбус,65,16250,31200.0,-14950.0
6,подписка на suppi-блог,33,4950,10560.0,-5610.0


## Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit'. Средняя прибыль с заказа

In [490]:
df.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [491]:
order_profit = []

In [492]:
for i in range(len(df)):
    products = df.loc[i]['products']
    warehouse = df.loc[i]['warehouse_name']
    income = sum(map(lambda x: x['price'] * x['quantity'], products))
    expenses = sum([i['quantity'] * answer1_dict[warehouse] for i in products])
    order_profit.append(income - expenses)

In [493]:
dict3 = {
    'order_id': df['order_id'],
    'order_profit': order_profit
}

In [494]:
answer_3_df = pd.DataFrame(dict3)

### Ответ

In [495]:
answer_3_df

,order_id,order_profit
0,11973,3980.0
1,62239,985.0
2,85794,-30.0
3,33684,1980.0
4,25824,975.0
...,...,...
95,28106,320.0
96,98100,1590.0
97,79293,1175.0
98,2930,720.0


In [496]:
asnwer_3_avg = answer_3_df['order_profit'].mean()

In [497]:
asnwer_3_avg

1352.3

## Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

In [604]:
warehouse_name_list = []
product_list = []
quantity_list = []
profit_list = []

dict4 = {
    'warehouse_name': [],
    'product': [],
    'quantity': [],
    'profit': [],
}

In [605]:
df.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,70,"[{'product': 'ломтик июльского неба', 'price':..."
30,63213,Мордор,30,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,30,"[{'product': 'билет в Израиль', 'price': 1000,..."
87,26702,Мордор,40,"[{'product': 'статуэтка Ленина', 'price': 200,..."
17,27443,Мордор,10,"[{'product': 'статуэтка Ленина', 'price': 200,..."


In [606]:
df = df.sort_values(by='warehouse_name')

In [607]:
df

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,70,"[{'product': 'ломтик июльского неба', 'price':..."
98,2930,Мордор,30,"[{'product': 'плюмбус', 'price': 250, 'quantit..."
33,56905,Мордор,20,"[{'product': 'билет в Израиль', 'price': 1000,..."
48,67495,Мордор,70,"[{'product': 'подписка на suppi-блог', 'price'..."
41,83618,Мордор,30,"[{'product': 'зеленая пластинка', 'price': 10,..."
...,...,...,...,...
26,99220,хутор близ Диканьки,75,"[{'product': 'плюмбус', 'price': 250, 'quantit..."
77,64268,хутор близ Диканьки,75,"[{'product': 'автограф Стаса Барецкого', 'pric..."
23,85787,хутор близ Диканьки,90,"[{'product': 'зеленая пластинка', 'price': 10,..."
42,57775,хутор близ Диканьки,30,"[{'product': 'статуэтка Ленина', 'price': 200,..."


In [608]:
df.loc[30]['products']

[{'product': 'зеленая пластинка', 'price': 10, 'quantity': 3}]

In [609]:
for i in df.index:
    warehouse = df.loc[i]['warehouse_name']
    warehouse_name_list.append(warehouse)
    
    products = df.loc[i]['products']
    product_list.append(', '.join(map(lambda x: x['product'], products)))
    
    income = sum(map(lambda x: x['price'] * x['quantity'], products))
    expenses = sum([i['quantity'] * answer1_dict[warehouse] for i in products])
    profit = income - expenses
    profit_list.append(profit)
    
    quantity = sum(map(lambda x: x['quantity'], products))
    quantity_list.append(quantity)

In [610]:
dict4 = {
    'warehouse_name': warehouse_name_list,
    'product': product_list,
    'quantity': quantity_list,
    'profit': profit_list
}

In [611]:
df4 = pd.DataFrame(dict4)

In [612]:
df4

,warehouse_name,product,quantity,profit
0,Мордор,"ломтик июльского неба, билет в Израиль, статуэ...",7,3980.0
1,Мордор,"плюмбус, плюмбус",3,720.0
2,Мордор,билет в Израиль,2,1980.0
3,Мордор,"подписка на suppi-блог, подписка на suppi-блог...",7,980.0
4,Мордор,зеленая пластинка,3,0.0
...,...,...,...,...
95,хутор близ Диканьки,"плюмбус, подписка на suppi-блог, плюмбус",5,1075.0
96,хутор близ Диканьки,"автограф Стаса Барецкого, зеленая пластинка, п...",5,985.0
97,хутор близ Диканьки,"зеленая пластинка, зеленая пластинка, билет в ...",6,960.0
98,хутор близ Диканьки,статуэтка Ленина,2,370.0


In [613]:
percent_profit_product_of_warehouse = []

In [614]:
profit_by_warehouse = df4.groupby('warehouse_name').sum()

C:\Users\dayyy\AppData\Local\Temp\ipykernel_2484\1757980370.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  profit_by_warehouse = df4.groupby('warehouse_name').sum()


In [625]:
for i in df4.index:
    cur_profit = df4.loc[i]['profit']
    cur_ware = df4.loc[i]['warehouse_name']
#     print(profit_by_warehouse.loc[cur_ware]['profit'])
    percent_profit_product_of_warehouse.append((cur_profit / profit_by_warehouse.loc[cur_ware]['profit']) * 100)

In [627]:
df4['percent_profit_product_of_warehouse'] = percent_profit_product_of_warehouse

### Ответ

In [629]:
df4

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,"ломтик июльского неба, билет в Израиль, статуэ...",7,3980.0,23.343109
1,Мордор,"плюмбус, плюмбус",3,720.0,4.222874
2,Мордор,билет в Израиль,2,1980.0,11.612903
3,Мордор,"подписка на suppi-блог, подписка на suppi-блог...",7,980.0,5.747801
4,Мордор,зеленая пластинка,3,0.0,0.000000
...,...,...,...,...,...
95,хутор близ Диканьки,"плюмбус, подписка на suppi-блог, плюмбус",5,1075.0,2.886681
96,хутор близ Диканьки,"автограф Стаса Барецкого, зеленая пластинка, п...",5,985.0,2.645005
97,хутор близ Диканьки,"зеленая пластинка, зеленая пластинка, билет в ...",6,960.0,2.577873
98,хутор близ Диканьки,статуэтка Ленина,2,370.0,0.993555


## Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться
'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [630]:
df5 = df4.copy()

In [634]:
df5 = df5.sort_values(by='percent_profit_product_of_warehouse', ascending=False)

In [635]:
df5

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,"ломтик июльского неба, билет в Израиль, статуэ...",7,3980.0,23.343109
68,отель Лето,"билет в Израиль, статуэтка Ленина, билет в Изр...",7,4425.0,16.358595
69,отель Лето,"зеленая пластинка, автограф Стаса Барецкого, б...",6,4060.0,15.009242
7,Мордор,"зеленая пластинка, плюмбус, автограф Стаса Бар...",8,2490.0,14.604106
45,остров невезения,"плюмбус, плюмбус, ломтик июльского неба",6,2070.0,13.260730
...,...,...,...,...,...
4,Мордор,зеленая пластинка,3,0.0,0.000000
13,Мордор,зеленая пластинка,3,0.0,0.000000
52,отель Лето,зеленая пластинка,1,-15.0,-0.055453
34,гиперборея,"зеленая пластинка, зеленая пластинка",4,-40.0,-0.104493


In [674]:
res5 = [df5.percent_profit_product_of_warehouse[0]]
lst = df5.percent_profit_product_of_warehouse.to_list()

In [675]:
for i in range(1, len(lst)):
    res5.append(lst[i] + sum(lst[:i]))

In [677]:
df5['accumulated_percent_profit_product_of_warehouse'] = res5

### Ответ

In [678]:
df5

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,Мордор,"ломтик июльского неба, билет в Израиль, статуэ...",7,3980.0,23.343109,23.343109
68,отель Лето,"билет в Израиль, статуэтка Ленина, билет в Изр...",7,4425.0,16.358595,39.701704
69,отель Лето,"зеленая пластинка, автограф Стаса Барецкого, б...",6,4060.0,15.009242,54.710946
7,Мордор,"зеленая пластинка, плюмбус, автограф Стаса Бар...",8,2490.0,14.604106,69.315051
45,остров невезения,"плюмбус, плюмбус, ломтик июльского неба",6,2070.0,13.260730,82.575782
...,...,...,...,...,...,...
4,Мордор,зеленая пластинка,3,0.0,0.000000,500.270852
13,Мордор,зеленая пластинка,3,0.0,0.000000,500.270852
52,отель Лето,зеленая пластинка,1,-15.0,-0.055453,500.215399
34,гиперборея,"зеленая пластинка, зеленая пластинка",4,-40.0,-0.104493,500.110906


## Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A.
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

In [696]:
bins = [0, 70, 91, 900]
labels = ['А', 'Б', 'С']

In [697]:
cuts = pd.cut(df5['accumulated_percent_profit_product_of_warehouse'], bins, labels=labels)

In [698]:
df5['category'] = cuts

### Ответ 

In [699]:
df5

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,Мордор,"ломтик июльского неба, билет в Израиль, статуэ...",7,3980.0,23.343109,23.343109,А
68,отель Лето,"билет в Израиль, статуэтка Ленина, билет в Изр...",7,4425.0,16.358595,39.701704,А
69,отель Лето,"зеленая пластинка, автограф Стаса Барецкого, б...",6,4060.0,15.009242,54.710946,А
7,Мордор,"зеленая пластинка, плюмбус, автограф Стаса Бар...",8,2490.0,14.604106,69.315051,А
45,остров невезения,"плюмбус, плюмбус, ломтик июльского неба",6,2070.0,13.260730,82.575782,Б
...,...,...,...,...,...,...,...
4,Мордор,зеленая пластинка,3,0.0,0.000000,500.270852,С
13,Мордор,зеленая пластинка,3,0.0,0.000000,500.270852,С
52,отель Лето,зеленая пластинка,1,-15.0,-0.055453,500.215399,С
34,гиперборея,"зеленая пластинка, зеленая пластинка",4,-40.0,-0.104493,500.110906,С
